### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [1]:
# imports
import warnings
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

# Disable warnings (including SSL warnings)
warnings.filterwarnings('ignore')

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please check it is set correctly in your .env file")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers, verify=False)  # verify=False (only for test purposes)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
an = Website("https://anthropic.com")
an.get_contents()

'Webpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nResearch\nEconomic Futures\nCommitments\nInitiatives\nTransparency\nResponsible Scaling Policy\nTrust center\nSecurity and compliance\nLearn\nLearn\nAnthropic Academy\nEngineering at Anthropic\nDeveloper docs\nCompany\nAbout\nCareers\nEvents\nNews\nTry Claude\nTry Claude\nTry Claude\nLearn more about Claude\nProducts\nClaude\nClaude Code\nClaude Developer Platform\nPricing\nContact sales\nModels\nOpus\nSonnet\nHaiku\nLog in\nClaude.ai\nClaude Console\nEN\nThis is some text inside of a div block.\nLog in to Claude\nLog in to Claude\nLog in to Claude\nDownload app\nDownload app\nDownload app\nResearch\nEconomic Futures\nCommitments\nInitiatives\nTransparency\nResponsible Scaling Policy\nTrust center\nSecurity and compliance\nLearn\nLearn\nAnthropic Academy\nEngineering at Anthropic\nDeveloper docs\nCompany\nAbout\nCareers\nEvents\nNews\nTry Claude\nTry Claude\nTry Claude\nLearn more about Claude\

In [5]:
an.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-futures',
 'https://www.anthropic.com/transparency',
 'https://www.anthropic.com/news/announcing-our-updated-responsible-scaling-policy',
 'http://trust.anthropic.com/',
 'https://www.anthropic.com/learn',
 'https://www.anthropic.com/engineering',
 'https://docs.claude.com',
 'https://www.anthropic.com/company',
 'https://www.anthropic.com/careers',
 'https://www.anthropic.com/events',
 'https://www.anthropic.com/news',
 'https://claude.ai',
 'https://claude.com/product/overview',
 'https://claude.com/product/claude-code',
 'https://claude.com/platform/api',
 'https://claude.com/pricing',
 'https://claude.com/contact-sales',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/claude/haiku',
 'https://claude.ai/login',
 'https://console.anthropic.com',
 '#',
 'https://claude.ai/login',
 'https://cla

## Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec.

In [28]:
# one-shot prompting for link extraction
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON format. Here are some examples:"
link_system_prompt += """Example 1:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""
link_system_prompt += """Example 2:
{
    "links": [
        {"type": "company page", "url": "https://techcorp.com/company"},
        {"type": "about page", "url": "https://techcorp.com/about-us"},
        {"type": "jobs page", "url": "https://techcorp.com/jobs"},
        {"type": "leadership page", "url": "https://techcorp.com/leadership"}
    ]
}
"""
link_system_prompt += """Example 3:
{
    "links": [
        {"type": "about page", "url": "https://startup.io/about"},
        {"type": "team page", "url": "https://startup.io/team"},
        {"type": "careers page", "url": "https://startup.io/work-with-us"}
    ]
}
"""

In [29]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON format. Here are some examples:Example 1:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
Example 2:
{
    "links": [
        {"type": "company page", "url": "https://techcorp.com/company"},
        {"type": "about page", "url": "https://techcorp.com/about-us"},
        {"type": "jobs page", "url": "https://techcorp.com/jobs"},
        {"type": "leadership page", "url": "https://techcorp.com/leadership"}
    ]
}
Example 3:
{
    "links": [
        {"type": "about page", "url": "https://startup.io/about"},
        {"type": "team page", "url": "https://startup.io/team"},
        {"type": "careers pa

In [30]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [31]:
print(get_links_user_prompt(an))

Here is the list of links on the website of https://anthropic.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#main
#footer
https://www.anthropic.com/
https://www.anthropic.com/research
https://www.anthropic.com/economic-futures
https://www.anthropic.com/transparency
https://www.anthropic.com/news/announcing-our-updated-responsible-scaling-policy
http://trust.anthropic.com/
https://www.anthropic.com/learn
https://www.anthropic.com/engineering
https://docs.claude.com
https://www.anthropic.com/company
https://www.anthropic.com/careers
https://www.anthropic.com/events
https://www.anthropic.com/news
https://claude.ai
https://claude.com/product/overview
https://claude.com/product/claude-code
https://claude.com/platform/api
https://claude.com/pricing
https://claude.com/contact-sales
https://www.anthropic.com/c

In [32]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"} # ensures we get valid JSON back
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [33]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/MiniMaxAI/MiniMax-M2',
 '/deepseek-ai/DeepSeek-OCR',
 '/meituan-longcat/LongCat-Video',
 '/PaddlePaddle/PaddleOCR-VL',
 '/tencent/HunyuanWorld-Mirror',
 '/models',
 '/spaces/khang119966/DeepSeek-OCR-DEMO',
 '/spaces/enzostvs/deepsite',
 '/spaces/Wan-AI/Wan2.2-Animate',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/merterbak/DeepSeek-OCR-Demo',
 '/spaces',
 '/datasets/HuggingFaceFW/finewiki',
 '/datasets/nvidia/PhysicalAI-Autonomous-Vehicles',
 '/datasets/karpathy/fineweb-edu-100b-shuffle',
 '/datasets/HuggingFaceM4/FineVision',
 '/datasets/nick007x/github-code-2025',
 '/datasets',
 '/join',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/inference/models',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/mi

In [34]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [35]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()  # get main page contents
    links = get_links(url)  # model call to get relevant links on page
    print("Found links:", links)
    # iterate through relevant links
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        # use the same Website class to get contents, append to result
        result += Website(link["url"]).get_contents()
    return result

In [36]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
MiniMaxAI/MiniMax-M2
Updated
about 18 hours ago
•
286k
•
752
deepseek-ai/DeepSeek-OCR
Updated
5 days ago
•
1.34M
•
2.23k
meituan-longcat/LongCat-Video
Updated
1 day ago
•
660
•
203
PaddlePaddle/PaddleOCR-VL
Updated
about 20 hours ago
•
22k
•
1.16k
tencent/HunyuanWorld-Mirror
Updated
5 days ago
•
18.3k
•
391
Browse 1M+ models
Spaces
Running
on
Zero
345
345
DeepSeek OCR Demo
🆘
An interactive demo for the DeepSeek-OCR model.
Running
15.5k
15.5k
DeepSite v3
🐳
Generate any application by Vibe Coding
Running
2.14k
2.14k
Wan2.2 Animate
👁
Wan2.2 Animate
Running
on
Zero
MCP
1.99k
1.99k
Wan2.2 14B Fast
🎥
generat

In [42]:
# use the results gathered to create a brochure
system_prompt = """You are an assistant that analyzes the contents of several relevant pages from a company website and creates a professional brochure about the company for prospective customers, investors and recruits.

INSTRUCTIONS:
- Respond in markdown format only
- Create a comprehensive but concise brochure (aim for 800-1200 words)
- Use professional, engaging language that appeals to all three audiences
- Include specific details and metrics when available
- Maintain consistent formatting and structure

REQUIRED BROCHURE STRUCTURE:

# [Company Name]
*One compelling tagline or mission statement*

## 🏢 Company Overview
Brief description of what the company does, when it was founded, and its primary focus/industry.

## 🎯 Mission & Vision
Company's mission statement, vision, and core values. What drives the organization?

## 💼 Products & Services
Key offerings, products, or services the company provides. Include target markets if available.

## 👥 Leadership & Team
Information about founders, key executives, team size, and company culture.

## 🌟 Why Choose Us
Unique value propositions, competitive advantages, key achievements, or notable clients/partnerships.

## 💰 Investment Highlights
(If applicable) Funding information, growth metrics, market opportunity, or financial highlights.

## 🚀 Career Opportunities
Available positions, company culture, benefits, growth opportunities, and what makes it a great place to work.

## 📈 Company Growth & Future
Recent milestones, future plans, expansion goals, or upcoming initiatives.

## 📞 Contact Information
How to get in touch for customers, investors, or job seekers.

FORMATTING GUIDELINES:
- Use emojis for section headers to make it visually appealing
- Use bullet points for lists
- Bold important keywords and metrics
- Include relevant statistics and numbers when available
- Keep paragraphs concise (2-4 sentences)
- Use subheadings within sections if needed

If information for any section is not available from the website content, skip that section entirely, rather than making up details."""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [43]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [44]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'team page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nMiniMaxAI/MiniMax-M2\nUpdated\nabout 18 hours ago\n•\n286k\n•\n752\ndeepseek-ai/DeepSeek-OCR\nUpdated\n5 days ago\n•\n1.34M\n•\n2.23k\nmeituan-longcat/LongCat-Video\nUpdated\n1 day ago\n•\n660\n•\n203\nPaddlePaddle/PaddleOCR-VL\nUpdated\nabout 21 hours ago\n•\n22k\n•\n1.16k\ntencent/HunyuanWorld-Mirror\nUpdated\n5 days ago\n•\n18.3k\n•\n391\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n34

In [48]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            # second LLM call to fetch contents via get_all_details()
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    return result

In [49]:
def display_brochure(text):
    display(Markdown(text))

In [50]:
brochure = create_brochure("HuggingFace", "https://huggingface.co")
display_brochure(brochure)

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'team page', 'url': 'https://huggingface.co/team'}, {'type': 'contact page', 'url': 'https://huggingface.co/contact'}]}


# Hugging Face
*The AI community building the future.*

## 🏢 Company Overview
Hugging Face is the leading platform for machine learning collaboration, founded with the vision of democratizing artificial intelligence through open-source resources. As a dynamic hub for models, datasets, and various ML applications, Hugging Face is dedicated to accelerating innovation in the AI community.

## 🎯 Mission & Vision
At Hugging Face, our **mission** is to foster collaboration within the machine learning ecosystem, enabling researchers, developers, and organizations to build and share advanced AI technologies seamlessly. Our **vision** is to create an inclusive community where everyone can contribute to and benefit from artificial intelligence. Our core values are:
- **Collaboration**: Empowering a community that shares knowledge and resources.
- **Transparency**: Commitment to open-source practices that promote accessibility.
- **Innovation**: Driving the future of AI through cutting-edge solutions.

## 💼 Products & Services
Hugging Face offers a suite of powerful tools and platforms designed for the AI community:
- **Models**: Access to **1M+ machine learning models**, enabling developers to experiment and innovate.
- **Datasets**: Explore **250k+ datasets** for various ML tasks, ensuring comprehensive data resources for every project.
- **Spaces**: A collaborative environment where users can showcase and run applications, currently featuring **400k+ applications**.
- **Enterprise Solutions**: Tailored offerings that include dedicated support, robust security, and advanced access control for organizations.

Target markets include researchers, academic institutions, startups, and large enterprises across various sectors such as tech, healthcare, and finance.

## 👥 Leadership & Team
Hugging Face’s team is composed of a diverse group of passionate individuals dedicated to advancing machine learning. Our leadership comprises seasoned professionals from both tech startups and established firms, reflecting a strong blend of experience and innovative thinking. The company culture emphasizes collaboration, creativity, and a commitment to open-source principles, fostering an environment where ideas can flourish.

## 🌟 Why Choose Us
Hugging Face stands out as an industry leader due to:
- **Community Engagement**: Over **65,761 active users** engaged in sharing knowledge and resources.
- **High-impact Collaborations**: Partnerships with notable organizations like Amazon, Google, and Microsoft, each utilizing our models and datasets.
- **Innovative Tools**: Best-in-class resources such as the **Transformers library**, with **over 151k models**, making it easier to deploy state-of-the-art AI.

## 💰 Investment Highlights
Although specific funding information is not available, Hugging Face is utilized by **more than 50,000 organizations**, highlighting significant market demand for its tools. The community base grows rapidly, indicating strong retention and engagement, which are essential metrics for future scaling and investment opportunities.

## 🚀 Career Opportunities
Hugging Face is on the lookout for talented individuals interested in shaping the future of AI. Positions span engineering, product, design, and community engagement roles. Our culture encourages:
- **Growth**: We provide training opportunities and career development pathways.
- **Well-being**: Competitive benefits including flexible working arrangements and health coverage.
- **Impact**: Work on pressing AI challenges that have the potential to transform industries globally.

## 📈 Company Growth & Future
Hugging Face continues to strive for innovation and rapid growth, with milestones including:
- Launch of cutting-edge tools like **Inference Endpoints**, optimized for seamless deployment.
- Ongoing expansion of the **Hugging Face Hub** to facilitate even more diverse use cases in AI.
- Plans for international outreach to foster global collaboration in the AI domain.

## 📞 Contact Information
For inquiries, collaborations, or career opportunities, please reach out to us:
- **Website**: [Hugging Face](https://huggingface.co)
- **Email**: contact@huggingface.co
- **Social Media**: Follow us on Twitter (@huggingface), LinkedIn, and GitHub for the latest updates and community engagement.

Join us at Hugging Face and become part of the vibrant AI community shaping the future together!

## Translate brochure to Spanish

In [51]:
# Translation system prompt for Spanish
translation_system_prompt = """You are a professional translator specializing in business and marketing content. 
Your task is to translate a company brochure from English to Spanish while maintaining:

TRANSLATION REQUIREMENTS:
- Professional, business-appropriate Spanish language
- Preserve all markdown formatting (headers, bullet points, emojis, etc.)
- Maintain the same structure and layout
- Use formal business Spanish appropriate for corporate communications
- Preserve company names, URLs, and proper nouns in their original form
- Keep technical terms accurate and industry-appropriate
- Ensure the translation appeals to Spanish-speaking customers, investors, and potential employees

FORMATTING RULES:
- Keep all markdown syntax exactly as provided
- Preserve emoji usage in headers
- Maintain bullet point formatting
- Keep bold and italic formatting intact
- Do not translate URLs or email addresses
- Company names should remain in English unless they have official Spanish versions

TONE AND STYLE:
- Use professional, engaging language
- Maintain the persuasive and informative tone of the original
- Use appropriate business terminology in Spanish
- Ensure cultural appropriateness for Spanish-speaking markets

Respond with only the translated markdown content, maintaining the exact same structure as the input."""

def translate_to_spanish(brochure_content):
    """
    Translates the brochure content to Spanish using OpenAI
    """
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translation_system_prompt},
            {"role": "user", "content": f"Please translate this company brochure to Spanish:\n\n{brochure_content}"}
        ],
        temperature=0.3  # Lower temperature for more consistent translations
    )
    return response.choices[0].message.content

In [55]:
def create_spanish_brochure(company_name, url):
    """
    Creates both English and Spanish versions of the brochure
    """
    # Get the English brochure
    english_brochure = create_brochure(company_name, url)
    
    # Translate to Spanish
    spanish_brochure = translate_to_spanish(english_brochure)
    return spanish_brochure

In [56]:
# Translate brochure to Spanish
spanish_brochure = create_spanish_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'team page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.lin

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [54]:
display_brochure(spanish_brochure)

# Hugging Face
*La comunidad de IA construyendo el futuro.*

## 🏢 Visión General de la Empresa
**Hugging Face** es una plataforma pionera que fomenta la colaboración dentro de la comunidad de aprendizaje automático (ML) al proporcionar herramientas para compartir modelos, conjuntos de datos y aplicaciones. Desde su creación, Hugging Face se ha convertido en una figura central en el panorama de la IA, con más de **1 millón de modelos** y ofreciendo una rica variedad de apoyo para desarrolladores e investigadores por igual.

## 🎯 Misión y Visión
Hugging Face tiene como objetivo democratizar la IA creando una plataforma abierta que promueva la **colaboración**, la **innovación** y el **compromiso comunitario**. Sus valores fundamentales incluyen:

- **Colaboración de Código Abierto**: Construir tecnología que esté disponible de forma gratuita y desarrollada de manera colaborativa.
- **Primero la Comunidad**: Priorizar las necesidades y contribuciones de la amplia comunidad de IA.
- **Enfoque en la Accesibilidad**: Hacer que las herramientas de IA sean accesibles para todos, independientemente de su nivel de experiencia.

## 💼 Productos y Servicios
Hugging Face ofrece numerosos productos y servicios adaptados para satisfacer las necesidades de diversas audiencias:

- **Modelos**: Acceso a una biblioteca diversa y en constante crecimiento de más de **1 millón de modelos** que cubren múltiples modalidades como texto, imagen, video, audio y 3D.
- **Conjuntos de Datos**: Una colección de más de **250,000 conjuntos de datos** para una variedad de tareas de ML.
- **Espacios**: Una plataforma interactiva para que los usuarios creen y muestren aplicaciones construidas sobre los modelos de Hugging Face.
- **Soluciones Empresariales**: Servicios personalizados que incluyen soporte prioritario, infraestructura dedicada y opciones de seguridad avanzadas para organizaciones.

**Mercados Objetivo**:
Hugging Face atiende a una amplia audiencia que incluye desarrolladores individuales, startups, instituciones académicas y grandes empresas como Google, Amazon y Microsoft.

## 👥 Liderazgo y Equipo
Hugging Face está dirigido por un apasionado grupo de expertos dedicados a avanzar en el campo de la IA. El equipo está compuesto por ingenieros, investigadores y educadores que fomentan una cultura vibrante de aprendizaje e innovación. La empresa prioriza un **entorno colaborativo e inclusivo**, alentando a todos a contribuir hacia el logro de objetivos colectivos.

## 🌟 Por Qué Elegirnos
Con una base de usuarios de más de **50,000 organizaciones**, Hugging Face se destaca por varias razones clave:

- **Recursos Extensos**: Una impresionante colección de **modelos de IA de última generación** y conjuntos de datos.
- **Innovación Impulsada por la Comunidad**: Una plataforma que empodera a los usuarios para construir, compartir y mejorar herramientas de ML colectivamente.
- **Características de Nivel Empresarial**: Ofreciendo soluciones que priorizan la seguridad, el control de acceso y el soporte integral.

Clientes notables incluyen organizaciones líderes como **Meta, Google y Microsoft**, lo que subraya la fiabilidad y las capacidades avanzadas de la plataforma.

## 💰 Aspectos Destacados de la Inversión
Hugging Face ha establecido una sólida presencia en el mercado de IA, demostrando métricas de crecimiento robustas. Sus soluciones empresariales los posicionan bien a medida que las organizaciones invierten cada vez más en tecnologías de IA. Al aprovechar un modelo de precios de **pago por uso** para recursos de computación, comenzando en **$0.60/hora por uso de GPU**, Hugging Face optimiza la gestión de costos para las empresas.

## 🚀 Oportunidades de Carrera
Hugging Face no solo es un líder en IA, sino también un gran lugar para trabajar. Las posiciones actuales abarcan diversas funciones, incluyendo ingeniería, gestión de comunidades y desarrollo de productos. Los beneficios de trabajar en Hugging Face incluyen:

- **Cultura Colaborativa**: Fomentando el trabajo en equipo interfuncional y el intercambio de conocimientos.
- **Oportunidades de Crecimiento**: Acceso a recursos de aprendizaje y desarrollo, mentoría y avance profesional.
- **Entorno Flexible**: Un enfoque en el equilibrio entre la vida laboral y personal y arreglos de trabajo flexibles.

## 📈 Crecimiento de la Empresa y Futuro
Hugging Face continúa experimentando un crecimiento e innovación significativos. Los hitos recientes incluyen:

- Lanzamiento de herramientas de vanguardia para el entrenamiento y despliegue de modelos.
- Expansión de la biblioteca de **herramientas de código abierto** para aprendizaje automático, que actualmente cuenta con más de **151,837 modelos Transformers**.
- Planes para mejorar la plataforma con características más intuitivas, mejorando la experiencia del usuario y la accesibilidad.

## 📞 Información de Contacto
Para consultas relacionadas con productos, servicios, oportunidades de inversión o carreras, comuníquese con Hugging Face en:

- **Sitio Web**: [huggingface.co](https://huggingface.co)
- **Correo Electrónico**: contact@huggingface.co
- **Redes Sociales**: Síguenos en [Twitter](https://twitter.com/huggingface) y [LinkedIn](https://www.linkedin.com/company/huggingface) para actualizaciones y compromiso comunitario.

---

¡Explora Hugging Face hoy y únete a una comunidad vibrante dedicada a construir el futuro de la IA!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    # add streaming interface
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## Welcome to Hugging Face – The AI Community Building the Future

Hugging Face is at the forefront of machine learning innovation, providing a collaborative platform where professionals, researchers, and enthusiasts from around the world can build, share, and explore state-of-the-art machine learning models and applications. 

---

## What We Do

At Hugging Face, we empower the AI community with tools that democratize artificial intelligence. Our platform hosts over **1 million models**, **250,000 datasets**, and a myriad of **interactive applications** through our Spaces feature. Whether you're looking to create, discover, or collaborate on machine learning projects, we've built the resources you need to succeed.

- **Models:** Access cutting-edge models for various tasks across text, image, audio, and more.
- **Datasets:** Explore thousands of datasets tailored for machine learning.
- **Spaces:** Create and share applications with ease.

---

## Our Customers

Join a diverse mix of organizations using Hugging Face to harness the power of AI:
- **Tech Giants**: Companies like Google, Microsoft, and Amazon utilize our platform for their advanced AI research and applications.
- **The Community**: These include academic teams and non-profit organizations dedicated to propelling AI innovation.

Over **50,000 organizations** trust us as their AI partner, demonstrating widespread confidence in our technology and community.

---

## Company Culture

At Hugging Face, we believe in an open, collaborative culture that encourages creativity and exploration. Our community-driven model fosters inclusivity, collaborative projects, and the sharing of knowledge across borders. We are committed to transparency and support a healthy work-life balance, ensuring all team members contribute to a mission-driven environment.

We recognize that the future of AI is built on a foundation of collaboration and transparency, and we actively promote these principles in everything we do.

---

## Careers and Opportunities

Hugging Face is expanding, and we’re always on the lookout for passionate individuals to join our team! By becoming part of our dynamic group, you'll contribute to the foundation of machine learning tooling and help shape the future of AI.

### Current Openings 
- **Software Engineers**
- **Machine Learning Researchers**
- **Community Managers**
- **Product Managers**

### Why Join Us?
- **Innovative Environment:** Work with cutting-edge technology and contribute to impactful projects.
- **Collaborative Culture:** Be part of a diverse team that encourages sharing insights and knowledge.
- **Growth Opportunities:** We support continuous learning and career advancement.

For potential applicants, you can find our current job openings on our [jobs page](https://huggingface.co/jobs).

---

## Join Us Today!

Become part of the revolution in AI development and collaboration. Whether you're looking to enhance your skills, partner on innovative projects, or join our growing team at Hugging Face, we invite you to explore our platform today.

- **Website:** [huggingface.co](https://huggingface.co)
- **Follow us on social media:** 
  - [Twitter](https://twitter.com/huggingface)
  - [LinkedIn](https://linkedin.com/company/huggingface)
  - [GitHub](https://github.com/huggingface)

Let's build the future together!

In [27]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Welcome to Hugging Face! 🤗

### Hugging Face – The AI Community Building the Future! 🌟

At Hugging Face, we’re not just a company; we’re a family. A tech-savvy, code-crunching, model-loving family! The place where machine learning enthusiasts and AI wizards unite to **build the future**. 🛠️✨

---

### What Do We Do? 🤔

We offer a *wonderland* where you can:

- Discover **1M+ AI Models** (No, that’s not a typo!)
- Access **250k+ Datasets** (Enough data to make even your big data dreams blush!)
- Create cutting-edge **AI Applications** in our magical **Spaces** (Think of them as cozy online coffee shops for coders).

**This week’s hottest models include:**

- **DeepSeek-OCR**: The model that sees everything! (Just kidding! Mostly text.)
- **MiniMax-M2**: Not just a movie title, but a ML superstar! 🍿
- And the ever-mysterious **LongCat-Video**! (We’re still investigating its origins... 🐱)

---

### Why Join the Hugging Face Family? 💖

**Culture of Collaboration**: 
- Here, the only competition is who can come up with the best meme about neural networks! 🤖😄
- We thrive on helping others grow, whether you’re in our office or halfway across the globe!

**For the Customers**:
- **Join over 50,000 organizations** using our platform! From Amazon to Google, we’re quite the popular kids on the block. We’re like the "it" model at the AI prom.
  
---

### Careers at Hugging Face: Your New Home! 🏠

Are you ready to embark on an adventure in AI? We’re on the lookout for bold explorers (that’s you!) eager to dive into Pytorch, help train transformer models, or pair up with SafetyTensor (a model with quite a bit of weight on its shoulders! 🏋️‍♂️).

- **Current Job Openings**: Check out our nifty jobs page because we want to hear from you! Don’t worry if you don’t understand everything – we hire for potential, not perfection. 😉

---

### Join the Community! 

Got an idea for a model that will solve world hunger, or maybe just generate cat memes? We have a **forum** for that! Connect with others, swap tips, and maybe challenge us to a dance-off – though we will not be responsible for any cringe! 💃

---

### Closing Remarks ✌️

Join us at Hugging Face where every day is an opportunity to learn, create, and maybe have a bit of fun along the way! Who says work can't be filled with laughter and AI sorcery? 

*Ready to embrace the future? Sign up today! We promise we’ll give you hugs, virtually speaking of course!*

---

📧 **Contact Us**: [Let’s Talk!](mailto:contact@huggingface.co)   

Follow us on:
- [GitHub](#) | [Twitter](#) | [LinkedIn](#) | [Discord](#)

**Hugging Face**: AI, but make it cozy! 🌈
